In [51]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.diagnostic import acorr_breusch_godfrey
from statsmodels.stats.stattools import jarque_bera
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV


In [52]:
newborn_train = pd.read_csv('../data/newborn_train_prep.csv')
newborn_test = pd.read_csv('../data/newborn_test_prep.csv')

In [53]:
newborn_train.head()

,Unnamed: 0,mother_marital_status,mother_height,mother_weight_gain,father_age,father_education,cigarettes_before_pregnancy,prenatal_care_month,number_prenatal_visits,previous_cesarean,newborn_gender,newborn_weight,body_mass_index_to_delivery_weight,mother_race_white,mother_race_black,mother_race_aian,mother_race_asian,mother_race_nhopi,mother_race_more_than_one
0,0,1.465098,0.302469,0.353446,-0.405828,0.999050,-0.223724,-0.217339,-0.361500,-0.430785,1.024157,3045,-0.219907,0.553114,-0.40252,-0.094952,-0.252175,-0.054113,-0.154076
1,2,-0.682548,0.675993,-0.053185,0.211499,0.999050,-0.223724,-0.150342,-0.109427,-0.430785,1.024157,3827,-0.002457,0.553114,-0.40252,-0.094952,-0.252175,-0.054113,-0.154076
2,3,-0.682548,-0.071056,-0.663131,-0.714491,0.999050,-0.223724,-0.150342,-0.613573,-0.430785,-0.976412,3997,0.533819,0.553114,-0.40252,-0.094952,-0.252175,-0.054113,-0.154076
3,4,-0.682548,0.302469,0.827848,-0.405828,2.146529,-0.223724,-0.217339,0.142646,-0.430785,1.024157,3240,-0.787252,0.553114,-0.40252,-0.094952,-0.252175,-0.054113,-0.154076
4,5,1.465098,-0.071056,1.437794,-0.714491,-0.722169,-0.223724,-0.083344,-0.361500,-0.430785,-0.976412,3544,-0.537870,0.553114,-0.40252,-0.094952,-0.252175,-0.054113,-0.154076


In [54]:
newborn_test.head()

,Unnamed: 0,mother_marital_status,mother_height,mother_weight_gain,father_age,father_education,cigarettes_before_pregnancy,prenatal_care_month,number_prenatal_visits,previous_cesarean,newborn_gender,body_mass_index_to_delivery_weight,mother_race_white,mother_race_black,mother_race_aian,mother_race_asian,mother_race_nhopi,mother_race_more_than_one
0,0,-0.683541,-0.444973,-0.458953,-0.252421,0.997547,-0.222923,-0.216920,0.897192,-0.430701,1.024038,0.272508,-1.80512,-0.40318,-0.094587,3.959254,-0.05397,-0.154326
1,1,-0.683541,1.052039,-0.458953,-0.560954,-0.722999,-0.222923,-0.082396,1.148246,-0.430701,1.024038,-0.304398,0.55398,-0.40318,-0.094587,-0.252573,-0.05397,-0.154326
2,2,1.462970,-0.070720,-0.663161,-1.486552,-1.296515,-0.222923,-0.216920,-0.107023,-0.430701,1.024038,0.320853,-1.80512,2.48028,-0.094587,-0.252573,-0.05397,-0.154326
3,3,1.462970,0.677786,-0.663161,-1.178020,-0.722999,-0.222923,-0.216920,0.395085,-0.430701,-0.976526,0.169977,0.55398,-0.40318,-0.094587,-0.252573,-0.05397,-0.154326
4,5,-0.683541,-1.193480,-1.207716,-0.252421,-0.722999,-0.222923,-0.149658,-0.107023,2.321798,-0.976526,1.698667,0.55398,-0.40318,-0.094587,-0.252573,-0.05397,-0.154326


In [55]:
newborn_train = newborn_train.drop('Unnamed: 0', axis=1)
newborn_test = newborn_test.drop('Unnamed: 0', axis=1)

# 5. Algorithm Selection
Now we move to choose the modeling techniques for regression. We decided to test following modeling techniques: Ordinary Least Squares, Elastic Net, Nearest Neighbors, Decision Tree, Support Vector Regression and then choose the technique that yield the best results. 
For every algorithm we will follow this steps: parameter tuning, create the model, train the model, test the model, iterate and improve

Linear regression is a simple and commonly used algorithm for regression tasks.
It assumes a linear relationship between the input features and the target variable.
We can use linear regression to model the relationship between the features in the dataset and the newborn weight.

### OLS
Ordinary Least Squares aims to minimize the sum of squared residuals between the predicted values and the actual values of the target variable. OLS assumes that the relationship between the independent variables and the dependent variable is linear and that the errors follow a normal distribution with constant variance. The estimated coefficients obtained from OLS represent the best linear unbiased estimates of the true population parameters. OLS is widely used due to its simplicity and interpretability, but it can be sensitive to outliers and violations of its underlying assumptions.

In [56]:
newborn_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708062 entries, 0 to 1708061
Data columns (total 18 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   mother_marital_status               float64
 1   mother_height                       float64
 2   mother_weight_gain                  float64
 3   father_age                          float64
 4   father_education                    float64
 5   cigarettes_before_pregnancy         float64
 6   prenatal_care_month                 float64
 7   number_prenatal_visits              float64
 8   previous_cesarean                   float64
 9   newborn_gender                      float64
 10  newborn_weight                      int64  
 11  body_mass_index_to_delivery_weight  float64
 12  mother_race_white                   float64
 13  mother_race_black                   float64
 14  mother_race_aian                    float64
 15  mother_race_asian                   float64
 16  

Now we will split our training sample into two parts: validation and train.

In [57]:
X_train, X_valid, y_train, y_valid = train_test_split(
    newborn_train.drop('newborn_weight', axis=1), newborn_train[['newborn_weight']], 
    test_size=0.25, random_state=3)



In [58]:
# add constant
X1_train = sm.add_constant(X_train)

# the OLS model
ols_model = sm.OLS(y_train, X1_train)
ols_results = ols_model.fit()

# print summary
ols_results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         newborn_weight   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     7304.
Date:                Sun, 11 Jun 2023   Prob (F-statistic):               0.00
Time:                        23:55:34   Log-Likelihood:            -9.9156e+06
No. Observations:             1281046   AIC:                         1.983e+07
Df Residuals:                 1281028   BIC:                         1.983e+07
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                               3282.5320      0.492   6678.514      0.000    3281.569    3283.495
mother_marital_status                -26.5009      0.746    -35.501      0.000     -27.964     -25.038
mother_height                         68.9507      0.699     98.680      0.000      67.581      70.320
mother_weight_gain                    81.6071      0.756    107.970      0.000      80.126      83.089
father_age                            -0.3304      0.524     -0.630      0.529      -1.358       0.697
father_education                      -4.2731      0.560     -7.626      0.000      -5.371      -3.175
cigarettes_before_pregnancy          -35.1309      0.500    -70.239      0.000     -36.111     -34.151
prenatal_care_month                  -13.9129      0.494    -28.175      0.000     -14.881     -12.945
number_prenatal_visits                64.6113      0.498    129.818      0.000      63.636      65.587
previous_cesarean                     19.0017      0.497     38.217      0.000      18.027      19.976
newborn_gender                       -56.9037      0.492   -115.723      0.000     -57.867     -55.940
body_mass_index_to_delivery_weight     3.5812      0.934      3.834      0.000       1.751       5.412
mother_race_white                  -5.992e+12   1.37e+13     -0.438      0.661   -3.28e+13    2.08e+13
mother_race_black                    -4.9e+12   1.12e+13     -0.438      0.661   -2.68e+13     1.7e+13
mother_race_aian                   -1.331e+12   3.04e+12     -0.438      0.661   -7.28e+12    4.62e+12
mother_race_asian                  -3.354e+12   7.65e+12     -0.438      0.661   -1.83e+13    1.16e+13
mother_race_nhopi                  -7.633e+11   1.74e+12     -0.438      0.661   -4.18e+12    2.65e+12
mother_race_more_than_one          -2.129e+12   4.86e+12     -0.438      0.661   -1.16e+13    7.39e+12
==============================================================================
Omnibus:                   195770.249   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           526117.733
Skew:                          -0.841   Prob(JB):                         0.00
Kurtosis:                       5.651   Cond. No.                     6.11e+13
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 7.62e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

We see that father_age is statistically insignificant so we will remove it.

In [59]:
# independent variables
X1_train = X1_train.drop('father_age', axis=1)

# the OLS model
ols_model = sm.OLS(y_train, X1_train)
ols_results = ols_model.fit()

# print summary
ols_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         newborn_weight   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     7760.
Date:                Sun, 11 Jun 2023   Prob (F-statistic):               0.00
Time:                        23:55:37   Log-Likelihood:            -9.9156e+06
No. Observations:             1281046   AIC:                         1.983e+07
Df Residuals:                 1281029   BIC:                         1.983e+07
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                               3282.5344      0.492   6677.150      0.000    3281.571    3283.498
mother_marital_status                -26.4585      0.736    -35.927      0.000     -27.902     -25.015
mother_height                         68.9302      0.698     98.748      0.000      67.562      70.298
mother_weight_gain                    81.6128      0.756    107.982      0.000      80.131      83.094
father_education                      -4.3309      0.553     -7.834      0.000      -5.414      -3.247
cigarettes_before_pregnancy          -35.1361      0.500    -70.261      0.000     -36.116     -34.156
prenatal_care_month                  -13.9158      0.494    -28.181      0.000     -14.884     -12.948
number_prenatal_visits                64.6090      0.498    129.819      0.000      63.634      65.584
previous_cesarean                     18.9635      0.494     38.418      0.000      17.996      19.931
newborn_gender                       -56.9042      0.492   -115.724      0.000     -57.868     -55.940
body_mass_index_to_delivery_weight     3.5694      0.934      3.823      0.000       1.739       5.400
mother_race_white                  -5.426e+12   1.37e+13     -0.396      0.692   -3.23e+13    2.14e+13
mother_race_black                  -4.438e+12   1.12e+13     -0.396      0.692   -2.64e+13    1.75e+13
mother_race_aian                   -1.206e+12   3.04e+12     -0.396      0.692   -7.17e+12    4.76e+12
mother_race_asian                  -3.038e+12   7.67e+12     -0.396      0.692   -1.81e+13     1.2e+13
mother_race_nhopi                  -6.913e+11   1.74e+12     -0.396      0.692   -4.11e+12    2.73e+12
mother_race_more_than_one          -1.928e+12   4.87e+12     -0.396      0.692   -1.15e+13    7.61e+12
==============================================================================
Omnibus:                   195832.367   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           526325.993
Skew:                          -0.841   Prob(JB):                         0.00
Kurtosis:                       5.651   Cond. No.                     6.09e+13
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 7.59e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

We can see that also variables connected to race are insignificant so we continue removing them one by one using the method from general to specific. At first we will remove mother_race_more_than_one because it concerns the most broad spectrum

In [60]:
# independent variables
X1_train = X1_train.drop('mother_race_more_than_one', axis=1)

# add constant
X1_train = sm.add_constant(X1_train)

# the OLS model
ols_model = sm.OLS(y_train, X1_train)
ols_results = ols_model.fit()

# print summary
ols_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         newborn_weight   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     8278.
Date:                Sun, 11 Jun 2023   Prob (F-statistic):               0.00
Time:                        23:55:39   Log-Likelihood:            -9.9156e+06
No. Observations:             1281046   AIC:                         1.983e+07
Df Residuals:                 1281030   BIC:                         1.983e+07
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                               3282.5301      0.491   6678.803      0.000    3281.567    3283.493
mother_marital_status                -26.6513      0.553    -48.209      0.000     -27.735     -25.568
mother_height                         68.9300      0.698     98.748      0.000      67.562      70.298
mother_weight_gain                    81.6119      0.756    107.982      0.000      80.131      83.093
father_education                      -4.3291      0.553     -7.831      0.000      -5.413      -3.246
cigarettes_before_pregnancy          -35.1365      0.500    -70.262      0.000     -36.117     -34.156
prenatal_care_month                  -13.9151      0.494    -28.180      0.000     -14.883     -12.947
number_prenatal_visits                64.6085      0.498    129.818      0.000      63.633      65.584
previous_cesarean                     18.9623      0.494     38.417      0.000      17.995      19.930
newborn_gender                       -56.9039      0.492   -115.724      0.000     -57.868     -55.940
body_mass_index_to_delivery_weight     3.5697      0.934      3.823      0.000       1.740       5.400
mother_race_white                     22.4082      1.390     16.117      0.000      19.683      25.133
mother_race_black                    -45.6315      1.210    -37.714      0.000     -48.003     -43.260
mother_race_aian                      10.5288      0.576     18.270      0.000       9.399      11.658
mother_race_asian                    -17.2011      0.912    -18.867      0.000     -18.988     -15.414
mother_race_nhopi                      4.1802      0.525      7.960      0.000       3.151       5.210
==============================================================================
Omnibus:                   195628.080   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           525552.786
Skew:                          -0.841   Prob(JB):                         0.00
Kurtosis:                       5.649   Cond. No.                         6.01
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Now all variables are significant so model is ready and we can interpret the results. R squared is 0.088 so our moedl explained 8.8% of the variation of target variable which is a bit small. All variables are jointly significant because p-value is 0 so we reject null hypothesis. Now we will perform tests to check other assumptions of linear regression.

In [61]:
# RESET Test
fitted_vals = ols_results.fittedvalues
reset_model = sm.OLS(ols_results.resid, sm.add_constant(np.column_stack((fitted_vals, fitted_vals**2))))
reset_results = reset_model.fit()
reset_test_statistic = reset_results.fvalue
reset_p_value = reset_results.f_pvalue

# Breusch-Pagan Test
bp_test_statistic, bp_p_value, _, _ = het_breuschpagan(ols_results.resid, X1_train)

# Breusch-Godfrey Test
bg_test_statistic, bg_p_value, _, _ = acorr_breusch_godfrey(ols_results, nlags=1)  # Specify the number of lags

# Jarque-Bera Test
jb_test_statistic, jb_p_value, _, _ = jarque_bera(ols_results.resid)


# Print the results
print("RESET Test:")
print("Test Statistic:", reset_test_statistic)
print("p-value:", reset_p_value)

print("\nBreusch-Pagan Test:")
print("Test Statistic:", bp_test_statistic)
print("p-value:", bp_p_value)

print("\nBreusch-Godfrey Test:")
print("Test Statistic:", bg_test_statistic)
print("p-value:", bg_p_value)

print("\nJarque-Bera Test:")
print("Test Statistic:", jb_test_statistic)
print("p-value:", jb_p_value)

RESET Test:
Test Statistic: 699.2044776825902
p-value: 3.1987016899408065e-304

Breusch-Pagan Test:
Test Statistic: 16002.504230868584
p-value: 0.0

Breusch-Godfrey Test:
Test Statistic: 5.584146263252677
p-value: 0.018123765307410385

Jarque-Bera Test:
Test Statistic: 525552.7862625656
p-value: 0.0


For Breusch-Pagan, Breusch-Godfrey and Jarque-Bera tests p-values are 0 (or close to zero) so we rejest null hypothesis, so we have heteroscedasticity, there is autocorrelation and residuals aren't normally distributed. Only p-value for RESET Test is higher than significance level so we conclude that model have linear form.

According to really small rsquared and that model does not pass many tests, he is not the best choice for prediting newborn weight.

Now we will train our model using our training set X_train and y_train.

Now we can evaluate the model performance by comparing its predictions with the actual true values in y_valid using the MAPE metric.

In [62]:
y1_valid = newborn_train['newborn_weight']
X1_valid = newborn_train[['mother_marital_status', 'mother_height', 'mother_weight_gain', 'father_education', 'cigarettes_before_pregnancy', 'prenatal_care_month', 'number_prenatal_visits', 'previous_cesarean', 'newborn_gender', 'body_mass_index_to_delivery_weight', 'mother_race_white', 'mother_race_black', 'mother_race_aian', 'mother_race_asian', 'mother_race_nhopi']]
X1_valid = sm.add_constant(X1_valid)

In [63]:
# Predict the target variable 
y1_pred_v = ols_results.predict(X1_valid)

# Calculate the mean absolute percentage error
ols_mape = mean_absolute_percentage_error(y1_valid, y1_pred_v)
print("MAPE =", ols_mape)

MAPE = 0.15791843365139757


This means that, on average, the predictions from the model have an absolute percentage error of around 15.79%.

The MAPE provides a measure of the accuracy of the predictions relative to the actual values. A lower MAPE indicates better prediction performance. In this case, a MAPE of 0.1579 suggests that the linear regression model has reasonably good predictive power for the target variable in the test set.


### Elastic Net

Elastic Net is a combination of Ridge regression and Lasso regression. Like Ridge regression, it adds a penalty term to the linear regression objective function. However, in Elastic Net, the penalty term consists of a combination of the L1 norm (sum of the absolute values of the coefficients) and the L2 norm (sum of the squared values of the coefficients), weighted by two regularization parameters: alpha and l1_ratio. The alpha parameter controls the overall strength of regularization, while the l1_ratio determines the balance between L1 and L2 regularization. Elastic Net can be useful when there are many correlated features and the model needs to perform feature selection and regularization simultaneously. 

We will use scikit-learn's GridSearchCV to perform a grid search with cross-validation for hyperparameter tuning of the ElasticNet model. The parameter_space dictionary defines the range of values to explore for each hyperparameter.

In [ ]:
X2_train=X_train
y2_train=y_train

parameter_space = {
    "alpha": [1, 10, 100, 200, 300],
    "l1_ratio": [0.5, 1],
    "fit_intercept": [True, False],
}

clf = GridSearchCV(ElasticNet(random_state=3), parameter_space, 
                   n_jobs=4, cv=3, scoring="neg_mean_absolute_error")

clf.fit(X2_train, y2_train)
print("Best parameters:")
print(clf.best_params_)

Best parameters:
{'alpha': 1, 'fit_intercept': True, 'l1_ratio': 1}


The n_jobs parameter is set to 3, indicating that the grid search will use three parallel jobs for training and evaluating the models. This means that three separate processes will be spawned to perform the computations concurrently, utilizing the available CPU cores. 

The cv parameter is set to 3, indicating that a 3-fold cross-validation strategy will be used during the grid search. This means the data will be split into three subsets, and the model will be trained and evaluated three times, each time using a different subset as the validation set. The performance metric specified by the scoring parameter (neg_mean_absolute_error in this case) will be used to evaluate the models and select the best hyperparameters.

After the grid search is complete, the best hyperparameters are printed using clf.best_params_. These are the hyperparameters that yielded the best performance according to the chosen scoring metric.

The alpha parameter controls the regularization strength of the model. A smaller value indicates weaker regularization. In this case, the best value found is 1.

The fit_intercept parameter determines whether or not an intercept term should be included in the model. The best value found is True, indicating that including an intercept term improves the model's performance.

The l1_ratio parameter indicating a stronger preference for L1 (Lasso) regularization. 


In [ ]:
# create an instance of the ElasticNet
elasticNet_model = ElasticNet(random_state=3, **clf.best_params_)

In [ ]:
# fitting the model 
elasticNet_model.fit(X2_train, y2_train)

ElasticNet(alpha=1, l1_ratio=1, random_state=3)

In [ ]:
X2_valid=X_valid
y2_valid=y_valid

# Predict the target variable
y2_pred_v = elasticNet_model.predict(X2_valid)

# Calculate the mean absolute percentage error
elasticNet_mape = mean_absolute_percentage_error(y2_valid, y2_pred_v)
print("MAPE =", elasticNet_mape)

MAPE = 0.1581801353160445


This means that, on average, the predictions from the model have an absolute percentage error of around 15.81%.

### Nearest Neighbors

We will apply k-Nearest Neighbors (k-NN) which is a non-parametric algorithm that makes predictions based on the similarity of data points in the feature space.

We will use parameter_space dictionary to define the hyperparameter options for the KNN regression model, including n_neighbors (number of neighbors to consider), weights (weighting scheme for neighbors), algorithm (algorithm used to compute nearest neighbors), and leaf_size (leaf size for tree-based algorithms).

In [ ]:
X3_train=X_train
y3_train=y_train
X3_valid=X_valid
y3_valid=y_valid

# Define the parameter space
parameter_space = {
    "n_neighbors": np.arange(3, 11),
    "weights": ["uniform", "distance"],
    "algorithm": ["ball_tree", "kd_tree", "brute"],
    "leaf_size": [1, 2, 10, 20, 50]
}

# Perform randomized search using KNeighborsRegressor
clf = RandomizedSearchCV(KNeighborsRegressor(), parameter_space, cv=3, 
                         scoring="neg_mean_absolute_error", n_jobs=3)

clf.fit(X3_train, y3_train)

print("Best parameters:")
print(clf.best_params_)


In [ ]:
knn_model = KNeighborsRegressor(**clf.best_params_)

In [ ]:
knn_model.fit(X3_train, y3_train)

In [ ]:
# Predict the target variable
y3_pred_v = knn_model.predict(X3_valid)

# Calculate the mean absolute percentage error
knn_mape = mean_absolute_percentage_error(y3_valid, y3_pred_v)
print("MAPE =", knn_mape)

### Decision Tree

To perform DT (Decision tree) we will use DecisionTreeRegressor class from scikit-learn.

In [ ]:
X4_train = X_train
y4_train = y_train
X4_valid = X_valid
y4_valid = y_valid

parameter_space = {
    "criterion": ["mse", "friedman_mse", "mae"],
    "min_samples_split": np.arange(3, 21),
    "min_samples_leaf": np.arange(3, 21),
    "max_features": np.arange(5, X4_train.shape[1] + 1),
}

clf = RandomizedSearchCV(DecisionTreeRegressor(random_state=3), parameter_space, 
                         cv=3, scoring="neg_mean_absolute_error", n_jobs=4)

clf.fit(X4_train, y4_train)

print("Best parameters:")
print(clf.best_params_)


In [ ]:
dt_model = DecisionTreeRegressor(**clf.best_params_)

In [ ]:
dt_model.fit(X4_train, y4_train)

In [ ]:
# Predict the target variable
y4_pred_v = dt_model.predict(X4_valid)

# Calculate the mean absolute percentage error
dt_mape = mean_absolute_percentage_error(y4_valid, y4_pred_v)
print("MAPE =", dt_mape)

### Support Vector Regression

Support Vector Regression (SVR) is a variant of Support Vector Machines (SVM) that can be used for regression tasks. SVR aims to find a hyperplane that best fits the training data while minimizing the margin violations.

We will search for the best model parameters using GridSearchCV(). Kernel specifies the kernel type, degree - the degree of the polynomial kernel, poly and gamma - the kernel coefficient for rbf, poly, sigmoid, coef0 - is independent term in kernel function and C is the penalty parameter of the error term.

In [ ]:
X5_train=X_train
y5_train=y_train
X5_valid=X_valid
y5_valid=y_valid

parameter_space = \
    {
        "kernel": ["poly", "linear", "rbf", "sigmoid"],
        "degree": [3, 5],
        "coef0": [0, 3, 7],
        "gamma":[1e-3, 1e-1, 1/X5_train.shape[1]],
        "C": [1, 10, 100],
    }

clf = GridSearchCV(SVR(), parameter_space, cv=3, n_jobs=4,
                   scoring="neg_mean_absolute_error")

clf.fit(X5_train, y5_train)
print("Best parameters:")
print(clf.best_params_)

In [ ]:
svr_model = SVR(**clf.best_params_)

In [ ]:
svr_model.fit(X5_train, y5_train)

In [ ]:
# Predict the target variable
y5_pred_v = svr_model.predict(X5_valid)

# Calculate the mean absolute percentage error
svr_mape = mean_absolute_percentage_error(y5_valid, y5_pred_v)
print("MAPE =", svr_mape)

### Random Forest

In [ ]:
X6_train=X_train
y6_train=y_train
X6_valid=X_valid
y6_valid=y_valid

parameter_space = \
    {
        "n_estimators": [10, 100, 300, 600],
        "criterion": ["mse", "mae"],
        "max_depth": [7, 50, 254],
        "min_samples_split": [2, 5],
        "min_samples_leaf": [1, 5],
        "max_features": [19, 100, X6_train.shape[1]],
        "bootstrap": [True, False],
    }

clf = RandomizedSearchCV(RandomForestRegressor(random_state=3), 
                         parameter_space, cv=3, n_jobs=4,
                         scoring="neg_mean_absolute_error", 
                         n_iter=10, random_state=3)

clf.fit(X6_train, y6_train)
print("Best parameters:")
print(clf.best_params_)

In [ ]:
rf_model = RandomForestRegressor(**clf.best_params_)

In [ ]:
rf_model.fit(X6_train, y6_train)

In [ ]:
y6_pred_v = rf_model.predict(X6_valid)
rf_mape = mean_absolute_percentage_error(y6_valid, y6_pred_v)
print("MAPE =", rf_mape)

### XGBoost model

In [ ]:
X7_train=X_train
y7_train=y_train
X7_valid=X_valid
y7_valid=y_valid

parameter_space = \
    {
        "max_depth": [4, 5, 6],
        "learning_rate": [0.005, 0.009, 0.01],
        "n_estimators": [700, 1000, 2500],
        "booster": ["gbtree",],
        "gamma": [7, 25, 100],
        "subsample": [0.3, 0.6],
        "colsample_bytree": [0.5, 0.7],
        "colsample_bylevel": [0.5, 0.7,],
        "reg_alpha": [1, 10, 33],
        "reg_lambda": [1, 3, 10],
    }

clf = RandomizedSearchCV(XGBRegressor(random_state=3), 
                         parameter_space, cv=3, n_jobs=4,
                         scoring="neg_mean_absolute_error", 
                         random_state=3, n_iter=10)

clf.fit(X7_train, y7_train)
print("Best parameters:")
print(clf.best_params_)

In [ ]:
xgb_model = XGBRegressor(**clf.best_params_)

In [ ]:
xgb_model.fit(X7_train, y7_train)

In [ ]:
y7_pred_v = xgb_model.predict(X7_valid)
xgb_mape = mean_absolute_percentage_error(y7_valid, y7_pred_v)
print("MAPE =", xgb_mape)

# Conclusion

We created five models and for every model we calculated MAPE. We are searching for the smallest value of MAPE. So we will choose the model to predict for the model with the smallest MAPE.